In [5]:
# change these to try this notebook out# chang 
BUCKET = 'atos-asl'
PROJECT = 'qwiklabs-gcp-aebfb78fe0f1b1d1'
REGION = 'us-central1'

In [6]:
import os
import tensorflow as tf
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.8'


In [7]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [7]:
len([[0.0] for i in range(0, 24)])

24

In [64]:
%%bash
mkdir ./train
touch ./train/__init__.py

In [71]:
%%writefile ./train/model.py
#!/usr/bin/env python

# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

TIMESERIES_COL = 'price'
N_OUTPUTS = 1  # in each sequence, 1-49 are features, and 50 is label
SEQ_LEN = None
DEFAULTS =None
N_INPUTS = None

CSV_COLUMNS = [ 'prediction_date',
               'avg_wind_speed_100m',
               'avg_wind_direction_100m',
               'avg_temperature',
               'avg_air_density',
               'avg_pressure',
               'avg_precipitation',
               'avg_wind_gust',
               'avg_radiation',
               'avg_wind_speed',
               'avg_wind_direction',
               'price',
               'key'
              ]

LABEL_COLUMN = 'price'
NUMBER_OF_BUCKET = 42 # Number of week # Number of days: 289

# Set default values for each CSV column


def init(hparams):
    global SEQ_LEN, DEFAULTS, N_INPUTS
    SEQ_LEN = hparams['sequence_length']
    DEFAULTS = [[0.0] for x in range(0, SEQ_LEN)]
    N_INPUTS = SEQ_LEN - N_OUTPUTS


def linear_model(features, mode, params):
    X = features[TIMESERIES_COL]
    predictions = tf.layers.dense(X, 1, activation=None)
    return predictions


def dnn_model(features, mode, params):
    X = features[TIMESERIES_COL]
    h1 = tf.layers.dense(X, params['layers'].pop(0), activation=tf.nn.relu)
    
    for layer in params['layers']:
        h1 = tf.layers.dense(h1, layer, activation=tf.nn.relu)
        h1 = tf.layers.dropout(h1, params['dropout_rate'], training=(mode == tf.estimator.ModeKeys.TRAIN))
    
    predictions = tf.layers.dense(h1, 1, activation=None)  # linear output: regression
    return predictions

# cnn filter/ kernel_size / stride_size
def cnn_model(features, mode, params):
    X = tf.reshape(features[TIMESERIES_COL],
                   [-1, N_INPUTS, 1])  # as a 1D "sequence" with only one time-series observation (height)
    c1 = tf.layers.conv1d(X, filters=params['filter'],
                          kernel_size=params['kernel_size'], strides=params['stride_size'],
                          padding='same', activation=tf.nn.relu)
    
    p1 = tf.layers.max_pooling1d(c1, pool_size=2, strides=2)

    c2 = tf.layers.conv1d(p1, filters=params['filter'],
                          kernel_size=params['kernel_size'], strides=params['stride_size'],
                          padding='same', activation=tf.nn.relu)
    p2 = tf.layers.max_pooling1d(c2, pool_size=2, strides=2)
    
    outlen = p2.shape[1] * p2.shape[2]
    c2flat = tf.reshape(p2, [-1, outlen])
    
    if params['batch_norm']:
        h1 = tf.layers.dense(c2flat, 3, activation=None)
        h1 = tf.layers.batch_normalization(h1, training=(mode == tf.estimator.ModeKeys.TRAIN))
        h1 = tf.nn.relu(h1)
        
    else:
        h1 = tf.layers.dense(c2flat, 3, activation=tf.nn.relu)
    
    predictions = tf.layers.dense(h1, 1, activation=None)  # linear output: regression
    return predictions


def rnn_model(features, mode, params):
    # 1. dynamic_rnn needs 3D shape: [BATCH_SIZE, N_INPUTS, 1]
    x = tf.reshape(features[TIMESERIES_COL], [-1, N_INPUTS, 1])

    # 2. configure the RNN
    cell = tf.nn.rnn_cell.GRUCell(params['cell_size'] * 2)
    outputs, state = tf.nn.dynamic_rnn(cell, x, dtype=tf.float32)

    # 3. pass rnn output through a dense layer
    h1 = tf.layers.dense(state, N_INPUTS // 2, activation=tf.nn.relu)
    predictions = tf.layers.dense(h1, 1, activation=None)  # (?, 1)
    return predictions


# 2-layer RNN
def rnn2_model(features, mode, params):
    # dynamic_rnn needs 3D shape: [BATCH_SIZE, N_INPUTS, 1]
    x = tf.reshape(features[TIMESERIES_COL], [-1, N_INPUTS, 1])
   
    # 2. configure the RNN
    cell1 = tf.nn.rnn_cell.GRUCell(params['cell_size'] * 2)
    cell2 = tf.nn.rnn_cell.GRUCell(params['cell_size'] // 2)
    cells = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
    outputs, state = tf.nn.dynamic_rnn(cells, x, dtype=tf.float32)
    # 'state' is now a tuple containing the final state of each cell layer
    # we use state[1] below to extract the final state of the final layer
    
    # 3. pass rnn output through a dense layer
    h1 = tf.layers.dense(state[1], cells.output_size // 2, activation=tf.nn.relu)
    predictions = tf.layers.dense(h1, 1, activation=None)  # (?, 1)
    return predictions


# create N-1 predictions
def rnnN_model(features, mode, params):
    # dynamic_rnn needs 3D shape: [BATCH_SIZE, N_INPUTS, 1]
    x = tf.reshape(features[TIMESERIES_COL], [-1, N_INPUTS, 1])

    # 2. configure the RNN
    cell1 = tf.nn.rnn_cell.GRUCell(params['cell_size'] * 2)
    cell2 = tf.nn.rnn_cell.GRUCell(params['cell_size'] // 2)
    cells = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
    outputs, state = tf.nn.dynamic_rnn(cells, x, dtype=tf.float32)
    # 'outputs' contains the state of the final layer for every time step
    # not just the last time step (?,N_INPUTS, final cell size)
    
    # 3. pass state for each time step through a DNN, to get a prediction
    # for each time step 
    h1 = tf.layers.dense(outputs, cells.output_size, activation=tf.nn.relu)
    h2 = tf.layers.dense(h1, cells.output_size // 2, activation=tf.nn.relu)
    predictions = tf.layers.dense(h2, 1, activation=None)  # (?, N_INPUTS, 1)
    predictions = tf.reshape(predictions, [-1, N_INPUTS])
    return predictions # return prediction for each time step


# read data and convert to needed format
def read_dataset(filename, mode, batch_size=512):
    def _input_fn():
        def decode_csv(row):
            # row is a string tensor containing the contents of one row
            features = tf.decode_csv(row, record_defaults=DEFAULTS)  # string tensor -> list of 50 rank 0 float tensors
            label = features.pop()  # remove last feature and use as label
            features = tf.stack(features)  # list of rank 0 tensors -> single rank 1 tensor
            return {TIMESERIES_COL: features}, label

        # Create list of file names that match "glob" pattern (i.e. data_file_*.csv)
        dataset = tf.data.Dataset.list_files(filename)
        # Read in data from files
        dataset = dataset.flat_map(tf.data.TextLineDataset)
        # Parse text lines as comma-separated values (CSV)
        dataset = dataset.map(decode_csv)

        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None  # loop indefinitely

        else:
            num_epochs = 1  # end-of-input after this

        dataset = dataset.repeat(num_epochs).batch(batch_size)
        return dataset.make_one_shot_iterator().get_next()

    return _input_fn


def serving_input_fn():
    feature_placeholders = {
        TIMESERIES_COL: tf.placeholder(tf.float32, [None, N_INPUTS])
    }

    features = {
        key: tf.expand_dims(tensor, -1)
        for key, tensor in feature_placeholders.items()
    }
    features[TIMESERIES_COL] = tf.squeeze(features[TIMESERIES_COL], axis=[2])

    return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)


def compute_errors(features, labels, predictions):
    labels = tf.expand_dims(labels, -1)  # rank 1 -> rank 2 to match rank of predictions

    if predictions.shape[1] == 1:
        loss = tf.losses.mean_squared_error(labels, predictions)
        rmse = tf.metrics.root_mean_squared_error(labels, predictions)
        return loss, rmse
    else:
        # one prediction for every input in sequence
        # get 1-N of (x + label)
        labelsN = tf.concat([features[TIMESERIES_COL], labels], axis=1)
        labelsN = labelsN[:, 1:]
        # loss is computed from the last 1/3 of the series
        N = (2 * N_INPUTS) // 3
        loss = tf.losses.mean_squared_error(labelsN[:, N:], predictions[:, N:])
        # rmse is computed from last prediction and last label
        lastPred = predictions[:, -1]
        rmse = tf.metrics.root_mean_squared_error(labels, lastPred)
        return loss, rmse

# RMSE when predicting same as last value
def same_as_last_benchmark(features, labels):
    predictions = features[TIMESERIES_COL][:,-1] # last value in input sequence
    return tf.metrics.root_mean_squared_error(labels, predictions)


# create the inference model
def sequence_regressor(features, labels, mode, params):
    # 1. run the appropriate model
    model_functions = {
        'linear': linear_model,
        'dnn': dnn_model,
        'cnn': cnn_model,
        'rnn': rnn_model,
        'rnn2': rnn2_model,
        'rnnN': rnnN_model}
    model_function = model_functions[params['model']]
    predictions = model_function(features, mode, params)

    # 2. loss function, training/eval ops
    loss = None
    train_op = None
    eval_metric_ops = None
    if mode == tf.estimator.ModeKeys.TRAIN or mode == tf.estimator.ModeKeys.EVAL:
        loss, rmse = compute_errors(features, labels, predictions)

        if mode == tf.estimator.ModeKeys.TRAIN:
            # this is needed for batch normalization, but has no effect otherwise
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(update_ops):
                # 2b. set up training operation
                train_op = tf.contrib.layers.optimize_loss(
                    loss,
                    tf.train.get_global_step(),
                    learning_rate=tf.train.exponential_decay(learning_rate=params['learning_rate'],
                                                             global_step=tf.train.get_global_step(),
                                                             decay_steps=100,
                                                             decay_rate=0.95,
                                                             staircase=True),
                    optimizer="Adam")

        # 2c. eval metric
        eval_metric_ops = {
            "RMSE": rmse,
            "RMSE_same_as_last": same_as_last_benchmark(features, labels),
        }

    # 3. Create predictions
    if predictions.shape[1] != 1:
        predictions = predictions[:, -1]  # last predicted value
    predictions_dict = {"predicted": predictions}

    # 4. return EstimatorSpec
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=predictions_dict,
        loss=loss,
        train_op=train_op,
        eval_metric_ops=eval_metric_ops,
        export_outputs={
            'predictions': tf.estimator.export.PredictOutput(predictions_dict)}
    )


def train_and_evaluate(output_dir, hparams):
    get_train = read_dataset(hparams['train_data_path'],
                             tf.estimator.ModeKeys.TRAIN,
                             hparams['train_batch_size'])
    get_valid = read_dataset(hparams['eval_data_path'],
                             tf.estimator.ModeKeys.EVAL,
                             1000)
    estimator = tf.estimator.Estimator(model_fn=sequence_regressor,
                                       params=hparams,
                                       config=tf.estimator.RunConfig(
                                           save_summary_steps=50,
                                           save_checkpoints_secs=
                                           hparams['min_eval_frequency']),
                                       model_dir=output_dir)
    train_spec = tf.estimator.TrainSpec(input_fn=get_train,
                                        max_steps=hparams['train_steps'])
    exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
    eval_spec = tf.estimator.EvalSpec(input_fn=get_valid,
                                      steps=None,
                                      exporters=exporter,
                                      start_delay_secs=hparams['eval_delay_secs'],
                                      throttle_secs=hparams['min_eval_frequency'])
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)


Overwriting ./train/model.py


In [117]:
%%writefile ./train/task.py
# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Example implementation of code to run on the Cloud ML service.
"""

import traceback
import argparse
import json
import os
import tensorflow as tf
from . import model


if __name__ == '__main__':
  parser = argparse.ArgumentParser()
  # Input Arguments
  parser.add_argument(
      '--train_data_path',
      help='GCS or local path to training data',
      required=True
  )
  parser.add_argument(
      '--eval_data_path',
      help='GCS or local path to evaluation data',
      required=True
  )
  parser.add_argument(
      '--train_batch_size',
      help='Batch size for training steps',
      type=int,
      default=100
  )
  parser.add_argument(
      '--learning_rate',
      help='Initial learning rate for training',
      type=float,
      default=0.01
  )
  parser.add_argument(
      '--train_steps',
      help="""\
      Steps to run the training job for. A step is one batch-size,\
      """,
      type=int,
      default=0
  )
  parser.add_argument(
      '--sequence_length',
      help="""\
      This model works with fixed length sequences. 1-(N-1) are inputs, last is output
      """,
      type=int,
      default=24
  )
  parser.add_argument(
      '--output_dir',
      help='GCS location to write checkpoints and export models',
      required=True
  )
  model_names = [name.replace('_model','') \
                   for name in dir(model) \
                     if name.endswith('_model')]
  parser.add_argument(
      '--model',
      help='Type of model. Supported types are {}'.format(model_names),
      required=True
  )
  parser.add_argument(
      '--job-dir',
      help='this model ignores this field, but it is required by gcloud',
      default='junk'
  )
  parser.add_argument(
      '--eval_delay_secs',
      help='How long to wait before running first evaluation',
      default=10,
      type=int
  )
  parser.add_argument(
      '--min_eval_frequency',
      help='Minimum number of training steps between evaluations',
      default=60,
      type=int
  )
  parser.add_argument(
      '--dropout_rate',
      help='drop out rate between DNN layers, defaults to 0.00',
      default=0.00,
      type=float
  )
  parser.add_argument(
      '--layers',
      help='specifiy number of neurons in layers comma seperated',
      default='128,64,32',
      type=str
  )
  # cnn filter/ kernel_size / stride_size
  parser.add_argument(
      '--kernel_size',
      help='drop out rate between DNN layers, defaults to 0.00',
      default=3,
      type=int
  )
  parser.add_argument(
      '--filter',
      help='specifiy number of neurons in layers comma seperated',
      default=12,
      type=int
  )
  parser.add_argument(
      '--stride_size',
      help='specifiy number of neurons in layers comma seperated',
      default=3,
      type=int
  )
  # RNX cell size
  parser.add_argument(
      '--cell_size',
      help='specifiy number of neurons in layers comma seperated',
      default=3,
      type=int
  )
  parser.add_argument(
      '--batch_norm',
      help='flag to use batch normalization',
      default='False',
      type=str
  )
  
  args = parser.parse_args()
  hparams = args.__dict__
  
  hparams['batch_norm'] = hparams['batch_norm'] == 'True'
  # unused args provided by service
  hparams.pop('job_dir', None)
  hparams.pop('job-dir', None)
  
  hparams['layers'] = [int(layer) for layer in hparams['layers'].split(',')]
  output_dir = hparams.pop('output_dir')
  tf.logging.info('layers param parsed {}'.format(hparams['layers']))
  # Append trial_id to path if we are doing hptuning
  # This code can be removed if you are not using hyperparameter tuning
  output_dir = os.path.join(
      output_dir,
      json.loads(
          os.environ.get('TF_CONFIG', '{}')
      ).get('task', {}).get('trial', '')
  )

  # calculate train_steps if not provided
  if hparams['train_steps'] < 1:
     # 1,000 steps at batch_size of 100
     hparams['train_steps'] = (1000 * 100) // hparams['train_batch_size']
     print ("Training for {} steps".format(hparams['train_steps']))

  model.init(hparams)
  tf.logging.info(hparams)
  # Run the training job
  model.train_and_evaluate(output_dir, hparams)

Overwriting ./train/task.py


In [4]:
! rm -rf ../test

__init__.py  __init__.pyc  model.py  model.pyc	task.py


In [6]:
hparams = dict()
hparams['eval_delay_secs'] = 30
hparams['min_eval_frequency'] = 30
hparams['train_data_path'] = './data/timeserie_price_daily_train.csv'
hparams['eval_data_path'] = './data/timeserie_price_daily_eval.csv'
hparams['train_batch_size'] = 140
hparams['train_steps'] = 1000
hparams['model'] = 'rnnN'
hparams['learning_rate'] = 0.01
hparams['sequence_length'] = 24

init(hparams)
train_and_evaluate('../test', hparams)

NameError: name 'init' is not defined

In [9]:
%%bash
ls -lart $(pwd)/trained/*
rm -rf $(pwd)/trained/energy_forecast/*

total 1092
drwxr-xr-x 3 root root   4096 Sep 18 16:59 ..
drwxr-xr-x 2 root root   4096 Sep 18 17:00 eval
drwxr-xr-x 3 root root   4096 Sep 18 17:00 export
-rw-r--r-- 1 root root 296461 Sep 18 17:00 graph.pbtxt
-rw-r--r-- 1 root root    732 Sep 18 17:00 model.ckpt-0.index
-rw-r--r-- 1 root root  28064 Sep 18 17:00 model.ckpt-0.data-00000-of-00001
-rw-r--r-- 1 root root     81 Sep 18 17:00 checkpoint
-rw-r--r-- 1 root root 123293 Sep 18 17:00 model.ckpt-0.meta
drwxr-xr-x 4 root root   4096 Sep 18 17:00 .
-rw-r--r-- 1 root root 637726 Sep 18 17:00 events.out.tfevents.1537290041.dc5b91e37012


In [82]:
%bash
DATADIR=$(pwd)/data
OUTDIR=$(pwd)/traineeer/energy_forecast/dnn
rm -rf ${OUTDIR}
SEQ_LEN=24
gcloud ml-engine local train \
   --module-name=train.task \
   --package-path=${PWD}/train \
   -- \
   --train_data_path="${DATADIR}/timeserie_price_daily_train.csv" \
   --dropout_rate=0.1\
   --eval_data_path="${DATADIR}/timeserie_price_daily_eval.csv"  \
   --output_dir=${OUTDIR} \
   --batch_norm \
   --model=cnn \
   --train_steps=1000 \
   --sequence_length=$SEQ_LEN

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:layers param parsed [128, 64, 32]
INFO:tensorflow:{'layers': [128, 64, 32], 'cell_size': 3, 'dropout_rate': 0.1, 'min_eval_frequency': 60, 'learning_rate': 0.01, 'eval_delay_secs': 10, 'stride_size': 3, 'filter': 12, 'train_data_path': '/content/datalab/energy_forcasing/bram/data/timeserie_price_daily_train.csv', 'train_batch_size': 100, 'sequence_length': 24, 'model': 'cnn', 'batch_norm': True, 'train_steps': 1000, 'kernel_size': 3, 'eval_data_path': '/content/datalab/energy_forcasing/bram/data/timeserie_price_daily_eval.csv'}
INFO:tensorflow:TF_CONFIG environment variable: {u'environment': u'cloud', u'cluster': {}, u'job': {u'args': [u'--train_data_path=/con

In [99]:
%%bash
gsutil mkdir gs://${BUCKET}/bram/daily
gsutil -m rm -rf gs://${BUCKET}/bram/daily/*
gsutil -m cp data/timeserie_price_daily_*.csv gs://${BUCKET}/bram/daily

CommandException: The mb command requires a URL that specifies a bucket.
"gs://atos-asl/bram/daily" is not valid.
Removing gs://atos-asl/bram/daily/timeserie_price_daily_eval.csv#1537223524383379...
Removing gs://atos-asl/bram/daily/timeserie_price_daily_train.csv#1537223524393510...
/ [2/2 objects] 100% Done                                                       
Operation completed over 2 objects.                                              
Copying file://data/timeserie_price_daily_eval.csv [Content-Type=text/csv]...
Copying file://data/timeserie_price_daily_train.csv [Content-Type=text/csv]...
/ [2/2 files][ 36.5 KiB/ 36.5 KiB] 100% Done                                    
Operation completed over 2 objects/36.5 KiB.                                     


In [91]:
!echo $TFVERSION

In [85]:
%%bash
SEQ_LEN=24
TFVERSION='1.8'
for MODEL in linear dnn cnn rnn rnn2 rnnN; do
# for MODEL in cnn; do
  OUTDIR=gs://${BUCKET}/bram/energy_forecast/TEST/${MODEL}
  JOBNAME=bram_energy_forecast_${MODEL}_$(date -u +%y%m%d_%H%M%S)
  REGION=us-central1
  gsutil -m rm -rf $OUTDIR
  gcloud ml-engine jobs submit training $JOBNAME \
     --region=$REGION \
     --module-name=train.task \
     --package-path=${PWD}/train \
     --job-dir=$OUTDIR \
     --scale-tier=BASIC \
     --runtime-version=$TFVERSION \
     -- \
     --train_data_path="gs://${BUCKET}/bram/daily/*train*.csv" \
     --eval_data_path="gs://${BUCKET}/bram/daily/*eval*.csv"  \
     --output_dir=$OUTDIR \
     --train_steps=3000 \
     --learning_rate=0.1 \
     --dropout_rate=0.1 \
     --batch_norm \
     --sequence_length=$SEQ_LEN \
     --model=$MODEL
done

jobId: bram_energy_forecast_linear_180918_211348
state: QUEUED
jobId: bram_energy_forecast_dnn_180918_211353
state: QUEUED
jobId: bram_energy_forecast_cnn_180918_211358
state: QUEUED
jobId: bram_energy_forecast_rnn_180918_211402
state: QUEUED
jobId: bram_energy_forecast_rnn2_180918_211406
state: QUEUED
jobId: bram_energy_forecast_rnnN_180918_211411
state: QUEUED


CommandException: 1 files/objects could not be removed.
Job [bram_energy_forecast_linear_180918_211348] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe bram_energy_forecast_linear_180918_211348

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs bram_energy_forecast_linear_180918_211348
Removing gs://atos-asl/bram/energy_forecast/TEST/dnn/#1537294470863161...
Removing gs://atos-asl/bram/energy_forecast/TEST/dnn/checkpoint#1537294473191603...
Removing gs://atos-asl/bram/energy_forecast/TEST/dnn/eval/#1537294376826619...
Removing gs://atos-asl/bram/energy_forecast/TEST/dnn/eval/events.out.tfevents.1537294376.cmle-training-3183275879115884688#1537294476410717...
Removing gs://atos-asl/bram/energy_forecast/TEST/dnn/events.out.tfevents.1537294366.cmle-training-3183275879115884688#1537294474729928...
Removing gs://atos-asl/bram/energy_forecast/TEST/dnn/export/#153729

In [84]:
! gcloud ml-engine jobs describe bram_energy_forecast_rnnN_180918_211411

createTime: '2018-09-18T19:44:28Z'
endTime: '2018-09-18T19:52:45Z'
etag: R-ghNzeJNDc=
jobId: bram_energy_forecast_cnn_180918_194424
startTime: '2018-09-18T19:45:05Z'
state: SUCCEEDED
trainingInput:
  args:
  - --train_data_path=gs://atos-asl/bram/daily/*train*.csv
  - --eval_data_path=gs://atos-asl/bram/daily/*eval*.csv
  - --output_dir=gs://atos-asl/bram/energy_forecast/TEST/cnn
  - --train_steps=3000
  - --learning_rate=0.1
  - --dropout_rate=0.1
  - --batch_norm
  - --sequence_length=24
  - --model=cnn
  jobDir: gs://atos-asl/bram/energy_forecast/TEST/cnn
  packageUris:
  - gs://atos-asl/bram/energy_forecast/TEST/cnn/packages/f14cc1506c6cb9d42d70875657be73c8d1acc78370b59ea94d3ba480bb9c3842/train-0.0.0.tar.gz
  pythonModule: train.task
  region: us-central1
  runtimeVersion: '1.8'
trainingOutput:
  consumedMLUnits: 0.09

View job in the Cloud Console at:
https://console.cloud.google.com/ml/jobs/bram_energy_forecast_cnn_180918_194424?project=qwiklabs-gcp-aebfb78fe0f1b1d1

View logs at

In [9]:
from google.datalab.ml import TensorBoard
TensorBoard().start('gs://{}/bram/energy_forecast/TEST'.format(BUCKET))

TensorBoard was started successfully with pid 3986. Click here to access it.

3986

In [10]:
for pid in TensorBoard.list()['pid']:
  TensorBoard().stop(pid)
  print('Stopped TensorBoard with pid {}'.format(pid))

Stopped TensorBoard with pid 3986


In [88]:
TensorBoard().start('gs://{}/bram/energy_forecast/'.format(BUCKET))

TensorBoard was started successfully with pid 46034. Click here to access it.

46034

In [63]:
%%bash
rm ./data/timeserie_price_daily_train.csv
rm ./data/timeserie_price_daily_eval.csv
gsutil cp gs://atos-asl/datasets/timeserie_price_daily_train_scaled.csv ./data/timeserie_price_daily_train.csv
gsutil cp gs://atos-asl/datasets/timeserie_price_daily_eval_scaled.csv ./data/timeserie_price_daily_eval.csv
gsutil cp ./data/timeserie_price_daily_eval.csv gs://${BUCKET}/bram/daily/
gsutil cp ./data/timeserie_price_daily_train.csv gs://${BUCKET}/bram/daily/

Copying gs://atos-asl/datasets/timeserie_price_daily_train_scaled.csv...
/ [1 files][106.5 KiB/106.5 KiB]                                                
Operation completed over 1 objects/106.5 KiB.                                    
Copying gs://atos-asl/datasets/timeserie_price_daily_eval_scaled.csv...
/ [1 files][ 20.7 KiB/ 20.7 KiB]                                                
Operation completed over 1 objects/20.7 KiB.                                     
Copying file://./data/timeserie_price_daily_eval.csv [Content-Type=text/csv]...
/ [1 files][ 20.7 KiB/ 20.7 KiB]                                                
Operation completed over 1 objects/20.7 KiB.                                     
Copying file://./data/timeserie_price_daily_train.csv [Content-Type=text/csv]...
/ [1 files][106.5 KiB/106.5 KiB]                                                
Operation completed over 1 objects/106.5 KiB.                                    


In [49]:
# import pandas as pd
df = pd.read_csv('./data/timeserie_price_daily_train.csv')
print(df[df.isnull().any(axis=1)])
print(df.shape)
print(len(df.any().isna()))
df = df.dropna()
print(df.shape)
df[df.isna().any(axis=1)]

Empty DataFrame
Columns: [48.1, 42.27, 35.72, 35.13, 36.22, 32.4, 36.6, 43.1, 45.14, 45.14.1, 47.35, 47.35.1, 44.91, 48.1.1, 58.02, 61.01, 62.69, 58.15, 53.6, 47.34, 40.4, 36.0, 37.0, 37.98]
Index: []

[0 rows x 24 columns]
(222, 24)
24
(222, 24)


,48.1,42.27,35.72,35.13,36.22,32.4,36.6,43.1,45.14,45.14.1,...,58.02,61.01,62.69,58.15,53.6,47.34,40.4,36.0,37.0,37.98


In [94]:
%%bash
wc -l ./data/timeserie_price_daily_eval.csv
tail ./data/timeserie_price_daily_eval.csv

45 ./data/timeserie_price_daily_eval.csv
46.58,46.58,46.69,47.63,50.54,59.6,61.04,61.38,61.69,61.04,60.73,60.01,58.0,55.0,55.6,56.8,59.59,60.5,63.5,61.69,59.0,52.62,54.0,50.89
47.74,46.3,43.97,43.0,43.15,44.0,46.3,53.11,52.42,48.7,47.63,47.63,46.3,44.08,43.5,44.84,50.0,52.37,58.95,56.6,50.3,49.85,50.52,44.97
41.0,39.58,39.5,39.67,40.53,41.6,41.0,46.3,50.0,50.1,50.0,50.0,49.02,44.86,41.74,42.58,44.47,50.73,61.0,61.74,57.5,50.1,45.15,39.75
38.75,38.2,37.7,37.58,43.45,50.3,51.75,53.11,50.3,49.02,50.52,51.02,50.52,48.06,46.54,47.38,53.33,57.83,58.75,56.02,48.9,46.3,40.0,38.1
37.0,36.5,36.5,37.2,44.01,54.28,54.69,55.15,54.61,54.11,55.11,54.69,54.5,54.11,55.0,54.69,55.11,60.73,60.73,56.57,50.1,46.3,40.2,39.25
38.35,36.0,36.0,38.35,46.05,54.0,55.6,55.16,54.19,53.4,54.0,53.91,53.55,52.19,52.28,53.19,56.28,59.02,59.17,56.0,51.1,46.6,43.97,40.24
39.53,40.0,39.53,40.24,46.29,57.72,61.35,63.1,62.97,60.1,59.59,58.64,56.7,56.02,56.02,56.6,57.78,63.61,65.48,63.03,59.0,55.16,56.8,55.02
52.0,52.0,50.0,

In [120]:
%%writefile ./hyperparameter/dnn.yaml
trainingInput:
  scaleTier: BASIC_GPU
  hyperparameters:
    goal: MINIMIZE
    hyperparameterMetricTag: RMSE
    maxTrials: 30
    maxParallelTrials: 5
    enableTrialEarlyStopping: True
    params:
    - parameterName: train_batch_size
      type: INTEGER
      minValue: 8
      maxValue: 512
      scaleType: UNIT_LINEAR_SCALE
    - parameterName: dropout_rate
      type: DOUBLE
      minValue: 0.05
      maxValue: 0.25
      scaleType: UNIT_LINEAR_SCALE
    - parameterName: layers
      type: CATEGORICAL
      categoricalValues:
      - 256,128,64,32,16,8,4
      - 128,64,32
      - 128,32,16
      - 128,32,4
      - 64,16,2
      - 32,16,8

Overwriting ./hyperparameter/dnn.yaml


In [121]:
%%writefile ./hyperparameter/cnn.yaml
trainingInput:
  scaleTier: BASIC_GPU
  hyperparameters:
    goal: MINIMIZE
    hyperparameterMetricTag: RMSE
    maxTrials: 30
    maxParallelTrials: 5
    enableTrialEarlyStopping: True
    params:
    - parameterName: train_batch_size
      type: INTEGER
      minValue: 8
      maxValue: 512
      scaleType: UNIT_LINEAR_SCALE
    - parameterName: kernel_size
      type: INTEGER
      minValue: 2
      maxValue: 84
      scaleType: UNIT_LINEAR_SCALE
    - parameterName: filter
      type: INTEGER
      minValue: 2
      maxValue: 84
    - parameterName: stride_size
      type: INTEGER
      minValue: 2
      maxValue: 2
    - parameterName: batch_norm
      type: CATEGORICAL
      categoricalValues:
      - 'True'
      - 'False'

Overwriting ./hyperparameter/cnn.yaml


In [122]:
%%writefile ./hyperparameter/rnn.yaml
trainingInput:
  scaleTier: BASIC_GPU
  hyperparameters:
    goal: MINIMIZE
    hyperparameterMetricTag: RMSE
    maxTrials: 30
    maxParallelTrials: 5
    enableTrialEarlyStopping: True
    params:
    - parameterName: train_batch_size
      type: INTEGER
      minValue: 8
      maxValue: 512
      scaleType: UNIT_LINEAR_SCALE
    - parameterName: cell_size
      type: INTEGER
      minValue: 3
      maxValue: 84
      scaleType: UNIT_LINEAR_SCALE

Overwriting ./hyperparameter/rnn.yaml


In [103]:
!ln -s rnn.yaml ./hyperparameter/rnnN.yaml

In [104]:
!ls -l hyperparameter

total 12
-rw-r--r-- 1 root root 737 Sep 18 21:40 cnn.yaml
-rw-r--r-- 1 root root 642 Sep 18 21:31 dnn.yaml
lrwxrwxrwx 1 root root   8 Sep 18 21:55 rnn2.yaml -> rnn.yaml
lrwxrwxrwx 1 root root   8 Sep 18 21:55 rnnN.yaml -> rnn.yaml
-rw-r--r-- 1 root root 453 Sep 18 21:41 rnn.yaml


In [133]:
%%bash
SEQ_LEN=24
TFVERSION='1.8'
for MODEL in dnn cnn rnn rnn2 rnnN; do
# for MODEL in cnn; do
  OUTDIR=gs://${BUCKET}/bram/energy_forecast/hyperparameter/${MODEL}
  JOBNAME=bram_energy_forecast_${MODEL}_$(date -u +%y%m%d_%H%M%S)
  REGION=us-central1
  gsutil -m rm -rf $OUTDIR
  gcloud ml-engine jobs submit training $JOBNAME \
     --region=$REGION \
     --module-name=train.task \
     --package-path=${PWD}/train \
     --job-dir=$OUTDIR \
     --runtime-version=$TFVERSION \
     --config=./hyperparameter/${MODEL}.yaml \
     -- \
     --train_data_path="gs://${BUCKET}/bram/daily/*train*.csv" \
     --eval_data_path="gs://${BUCKET}/bram/daily/*eval*.csv"  \
     --output_dir=$OUTDIR \
     --model=$MODEL
done

jobId: bram_energy_forecast_dnn_180919_010213
state: QUEUED
jobId: bram_energy_forecast_cnn_180919_010221
state: QUEUED
jobId: bram_energy_forecast_rnn_180919_010227
state: QUEUED
jobId: bram_energy_forecast_rnn2_180919_010235
state: QUEUED
jobId: bram_energy_forecast_rnnN_180919_010243
state: QUEUED


Removing gs://atos-asl/bram/energy_forecast/hyperparameter/dnn/1/#1537309566978215...
Removing gs://atos-asl/bram/energy_forecast/hyperparameter/dnn/1/checkpoint#1537309569325940...
Removing gs://atos-asl/bram/energy_forecast/hyperparameter/dnn/1/eval/#1537309536960267...
Removing gs://atos-asl/bram/energy_forecast/hyperparameter/dnn/1/eval/events.out.tfevents.1537309537.cmle-training-master-49c51cf158-0-f6v5r#1537309572825894...
Removing gs://atos-asl/bram/energy_forecast/hyperparameter/dnn/1/events.out.tfevents.1537309525.cmle-training-master-49c51cf158-0-f6v5r#1537309571092729...
Removing gs://atos-asl/bram/energy_forecast/hyperparameter/dnn/1/export/#1537309539581890...
Removing gs://atos-asl/bram/energy_forecast/hyperparameter/dnn/1/export/exporter/#1537309539844142...
Removing gs://atos-asl/bram/energy_forecast/hyperparameter/dnn/1/export/exporter/1537309538/#1537309546447071...
Removing gs://atos-asl/bram/energy_forecast/hyperparameter/dnn/1/export/exporter/1537309538/saved_mode

In [12]:
from google.datalab.ml import TensorBoard
TensorBoard().start('gs://{}/bram/energy_forecast/hyperparameter'.format(BUCKET))

TensorBoard was started successfully with pid 4192. Click here to access it.

4192

In [11]:
for pid in TensorBoard.list()['pid']:
  TensorBoard().stop(pid)
  print('Stopped TensorBoard with pid {}'.format(pid))

KeyError: 'pid'

In [107]:
!gcloud ml-engine jobs stream-logs bram_energy_forecast_dnn_180918_220242

INFO	2018-09-18 22:02:45 +0000	service		Validating job requirements...
INFO	2018-09-18 22:02:46 +0000	service		Job creation request has been successfully validated.
INFO	2018-09-18 22:02:46 +0000	service		Job bram_energy_forecast_dnn_180918_220242 is queued.
^C


Command killed by keyboard interrupt



In [113]:
a = 0.051696656261924503
type(a)

float

In [135]:
import numpy as np

In [14]:
%%bash
MODEL_TYPE="rnn"
MODEL_NAME="energy_price_forecast"
MODEL_VERSION="ml_on_gcp"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/bram/energy_forecast/hyperparameter/rnn/6/export/exporter/ | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
# gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
# gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version $TFVERSION

Deleting and deploying energy_price_forecast ml_on_gcp from gs://atos-asl/bram/energy_forecast/hyperparameter/rnn/6/export/exporter/1537320039/ ... this will take a few minutes


This will delete version [ml_on_gcp]...

Do you want to continue (Y/n)?  Please enter 'y' or 'n':  Please enter 'y' or 'n':  Please enter 'y' or 'n':  
ERROR: (gcloud.ml-engine.versions.delete) FAILED_PRECONDITION: Field: name Error: Cannot delete the default version unless it is the only version in the model.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: Cannot delete the default version unless it is the only version in
      the model.
    field: name


In [139]:
from oauth2client.client import GoogleCredentials
import requests
import json

MODEL_NAME = 'energy_price_forecast'
MODEL_VERSION = 'ml_on_gcp'

token = GoogleCredentials.get_application_default().get_access_token().access_token
api = 'https://ml.googleapis.com/v1/projects/{}/models/{}/versions/{}:predict' \
         .format(PROJECT, MODEL_NAME, MODEL_VERSION)

print(api)
headers = {'Authorization': 'Bearer ' + token }
data = {
  'instances': [
    {
      'price': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,2]
    }
  ]
}
print(headers)
print(json.dumps(data))
response = requests.post(api, json=data, headers=headers)
print(response.content)

https://ml.googleapis.com/v1/projects/qwiklabs-gcp-aebfb78fe0f1b1d1/models/energy_price_forecast/versions/ml_on_gcp:predict
{'Authorization': u'Bearer ya29.c.ElodBoijvckfawky5emOPSZbxElQAKUQ-UJhTicBKSRk9SK8Fu6qRqakrQgW4DGixeUnxLvO0lMiSIM98mMHVm0vS7qfcShL-vltXejYg-RCVgEMkvMQDgqJTf0'}
{"instances": [{"price": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 2]}]}
{"predictions": [{"predicted": [4.311974048614502]}]}
